In [1]:
import sys
sys.path.append(r'E:/0code')
# sys.path.append(r'/home/wangyf226/0code')
# sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/pyml')

In [15]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pyml.feature_extraction.text import CountVectorizer
from pyml.linear_model.classification import LogisticClassifier
from pyml.neighbors.classification import KNeighborsClassifier
from pyml.metrics.classification import precision_score
from pyml.model_selection import KFold
from pyml.preprocessing import scale

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 读取数据集

1. 读取训练数据和测试数据为字符串的列表
2. 读取训练集label，并转换为数字格式

In [3]:
def read_train_text_to_list(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        contents = f.readlines()
    lines = [l.strip() for l in contents]
    return lines

In [4]:
train_ori_X = read_train_text_to_list('../data/trainData.txt')
train_ori_Y = read_train_text_to_list('../data/trainLabel.txt')
train_ori_Y = np.array([int(y) for y in train_ori_Y])
test_ori_X = read_train_text_to_list('../data/testData.txt')

In [36]:
train_ori_X = train_ori_X[:4000]
train_ori_Y = train_ori_Y[:4000]
test_ori_X = test_ori_X[:4000]

# 数据预处理 & 特征工程
1. Count Vectors as feature
2. TF-IDF Vectors as festures
3. Word Embeddings as features
4. Text/NLP based features
5. Topic Models as features

In [37]:
f_vier = CountVectorizer()
train_wc_X = f_vier.fit_transform(train_ori_X)
test_wc_X = f_vier.transform(test_ori_X)

In [38]:
train_X = scale(train_wc_X)
test_X = scale(test_wc_X)

In [39]:
train_Y = train_ori_Y

In [40]:
train_X.shape

(100, 4830)

# 交叉验证-寻找最好的参数

In [46]:
# clf = LogisticClassifier(learning_rate=0.001, num_iterations=30000)
# clf.fit(train_X, train_Y, watch=True)

In [47]:
k_range = range(1,21)
k_splits = 10
ms = KFold(k_splits=k_splits)
k_scores = np.zeros((len(k_range)))
for train_indices, test_indices in ms.split(train_X):
    for k in k_range:
        clf = KNeighborsClassifier(k=k)
        clf.fit(train_X[train_indices], train_Y[train_indices])
        y_pred = clf.predict(train_X[test_indices])
        score = precision_score(train_Y[test_indices], y_pred)
#         print('k : {} score: {}'.format(k, score))
        k_scores[k_range.index(k)] += score
avg_k_scores = k_scores/k_splits
print(avg_k_scores)
print("best k ", np.argmax(avg_k_scores)+1)

[0.56 0.56 0.54 0.55 0.54 0.55 0.54 0.48 0.57 0.52 0.51 0.5  0.54 0.53
 0.53 0.53 0.53 0.53 0.53 0.53]
best k  9


# 训练模型写入结果

In [ ]:
clf = KNeighborsClassifier(k=6)
clf.fit(train_X,

In [ ]:
y_pred = clf.predict(test_X)
sub = pd.DataFrame(y_pred)
sub.to_csv('../results/'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)